# 1. 터미널을 통해 스파크 서버의 접속 정보를 확인합니다
```bash
bash> cd ~/workspace/data-engineer-basic-training/day5
bash> docker-compose up -d
bash> docker-compose ps

# 아래와 같이 http://127.0.0.1 로 시작하는 URL 에서 127.0.0.1 부분을 현재 aws-instance-host 이름으로 변경합니다
bash> docker-compose logs notebook | grep http
...
or http://127.0.0.1:8888/?token=ad4f43203ac46f7f7f58807ab6781b1fd18b9ca5066664df
...
```

### 1-1. 웹 브라우저에서 <span style="color:blue"> "http://<호스트>.aiffelbiz.co.kr:8888/?token=<토큰정보>" </span> 에 접속합니다

### 아래의 노트북 화면에서 <span style="color:red"> [과제] </span> 가 없는 노트는 Shift + Enter 를 통해 수행하고, <span style="color:red"> [과제] </span> 가 있는 노트는 과제의 가이드를 따라 채워 수행하세요


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession \
    .builder \
    .appName("Data Engineer Basic Day5") \
    .config("spark.dataengineer.basic.day5", "step3") \
    .config("spark.sql.session.timeZone", "Asia/Seoul") \
    .getOrCreate()

In [ ]:
user = spark.read.parquet("user/20201025")
user.first()

# 2. 필요한 모든 데이터를 읽어서 데이터프레임으로 생성합니다

### 2-1. 각 경로에서 10/25 기준 데이터를 읽고 스키마와 데이터를 확인합니다
### <span style="color:red"> [과제.1] </span> 아래의 <span style="color:green"> 과제) </span> 코멘트에 명시된 내용을 참고하여 purchase25 테이블을 생성합니다

In [ ]:
# 1. 오늘은 2020/10/26 이고, 어제가 서비스 첫 날인 2020/10/25 날짜의 기준 지표를 생성합니다

user25 = spark.read.parquet("user/20201025")
user25.createOrReplaceTempView("user25") # user25 테이블생성
user25.printSchema() # 스키마 출력
user25.show(truncate=False) # 데이터 확인

purchase25 = spark.read.parquet("purchase/20201025")
# 과제 1) purchase25 테이블생성 구문 작성
purchase25.printSchema()
purchase25.show(truncate=False)

accesslog = spark.read.option("inferSchema", "true").json("access/20201025")
accesslog.createOrReplaceTempView("accesslog")

access25 = spark.sql("select a_id, a_tag, a_time, a_timestamp, a_uid from accesslog")
access25.createOrReplaceTempView("access25")
access25.printSchema()
access25.show(truncate=False)


### 2-2. 사용에 용이하게 하기 위해 2020/10/25 일자의 데이터만 필터하여 데이터프레임을 생성합니다
> unixtime 을 문자열 시간으로 변경하는 함수는 from_unixtime 입니다

### <span style="color:red"> [과제.2] </span> 아래의 <span style="color:green"> 과제) </span> 코멘트에 명시된 내용을 참고하여 각종 p_time_condition 조건문을 완성합니다

In [ ]:
# 과제 2) p_time_condition 값은 p_time 값이 '2020-10-25 00:00:00' 보다 크거나 같고, '2020-10-26 00:00:00' 보다 작은 조건입니다
p_time_condition = "p_time 컬럼의 범위를 지정하는 조건문을 완성합니다"

purchase = spark.sql("select from_unixtime(p_time) as p_time, p_uid, p_id, p_name, p_amount from purchase25").where(p_time_condition)
purchase.createOrReplaceTempView("purchase")
purchase.printSchema()
purchase.show(truncate=False)

u_signup_condition = "u_signup >= '20201025' and u_signup < '20201026'"

user = spark.sql("select u_id, u_name, u_gender from user25").where(u_signup_condition)
user.createOrReplaceTempView("user")
user.printSchema()
user.show(truncate=False)

access = spark.sql("select a_id, a_tag, a_timestamp, a_uid from access25")
access.createOrReplaceTempView("access")
access.printSchema()
access.show(truncate=False)

# 3. Daily Activer User 지표를 생성하세요

### 3-1. access 테이블을 이용하여 DAU 지표를 생성하세요
```text
지표정의 : 지정한 일자의 접속한 유저 수
지표산식 : 지정한 일자의 접속 테이블에 로그(로그인 혹은 로그아웃)가 한 번 이상 발생한 이용자의 빈도수
입력형태 : access 테이블
출력형태 : number
```

### <span style="color:red"> [과제.3] </span> 아래의 <span style="color:green"> 과제) </span> 코멘트에 명시된 내용을 참고하여 각종 distinctAccessUser 구문을 완성합니다

In [ ]:
spark.sql("select a_timestamp as a_time, a_uid from access").show(truncate=False)

# 과제 3) distinctAccessUser 는 access 테이블에 대해 'count(distinct column) as DAU' 명령을 이용하여 distinct user 지표를 생성합니다
distinctAccessUser = "select <유일값의 빈도수 함수> as DAU from access"

dau = spark.sql(distinctAccessUser)
dau.show(truncate=False)

# 4. Daily Paying User 지표를 생성하세요

### 4-1. purchase 테이블을 이용하여 PU 지표를 생성하세요
```text
지표정의 : 지정한 일자의 구매 유저 수
지표산식 : 지정한 일자의 구매 테이블에 한 번이라도 구매가 발생한 이용자의 빈도수
입력형태 : purchase 테이블
출력형태 : number
```

### <span style="color:red"> [과제.4] </span> 아래의 <span style="color:green"> 과제) </span> 코멘트에 명시된 내용을 참고하여 각종 distinctPayingUser 구문을 완성합니다

In [ ]:
spark.sql("select p_time, p_uid from purchase").show()

# 과제 4) distinctPayingUser 는 purchase 테이블에 대해 'count(distinct column) as PU' 함수를 통해 구매유저 수를 구합니다
distinctPayingUser = "select <유일값 빈도수 함수> as PU from purchase"

pu = spark.sql(distinctPayingUser)
pu.show()

# 5. Daily Revenue 지표를 생성하세요

### 5-1. purchase 테이블을 이용하여 PU 지표를 생성하세요
```text
지표정의 : 지정한 일자에 발생한 총 매출 금액
지표산식 : 지정한 일자의 구매 테이블에 저장된 전체 매출 금액의 합
입력형태 : access 테이블
출력형태 : number
```

### <span style="color:red"> [과제.5] </span> 아래의 <span style="color:green"> 과제) </span> 코멘트에 명시된 내용을 참고하여 각종 sumOfDailyRevenue 구문을 완성합니다

In [ ]:
spark.sql("select p_amount as daily_revenue from purchase").show()

# 과제 5) sumOfDailyRevenue 는 sum(column) as DR 함수를 통해 p_amount 의 합을 구하시오
sumOfDailyRevenue = "select <합계함수> as DR from purchase"

dr = spark.sql(sumOfDailyRevenue)
dr.show()

# 6. ARPU - Average Revenue Per User 지표를 생성하세요
> 이미 생성된 데이터프레임으로부터 이름으로 하나의 값을 가져오는 방법은 아래와 같습니다
```python
변수명 = 데이터프레임.collect()[0]["이름"]
v_dau = dau.collect()[0]["DAU"]
```


### 6-1. daily revenue 와 daily user 정보를 이용하여 ARPU 를 구하세요
```text
지표정의 : 유저 당 평균 발생 매출 금액
지표산식 : 총 매출 / 전체 유저 수 = DR / DAU
입력형태 : Daily Revenue, Daily Active User
출력형태 : number
```

### <span style="color:red"> [과제.6] </span> 아래의 <span style="color:green"> 과제) </span> 코멘트에 명시된 내용을 참고하여 각종 v_unknown / v_unknown 변수 값을 치환 합니다

In [ ]:
v_dau = dau.collect()[0]["DAU"]
v_pu = pu.collect()[0]["PU"]
v_dr = dr.collect()[0]["DR"]
v_unknown = 1

# 과제 6) ARPU (= DR / DAU) 계산을 위한 v_unknown 를 각 각 다른 변수로 치환하세요
print("+------------------+")
print("|             ARPU |")
print("+------------------+")
print("|        {} |".format(v_unknown / v_unknown))
print("+------------------+")

# 7. ARPPU - Average Revenue Per Paying User 지표를 생성하세요
> 이미 생성된 데이터프레임으로부터 이름으로 하나의 값을 가져오는 방법은 아래와 같습니다
```python
변수명 = 데이터프레임.collect()[0]["이름"]
v_dau = dau.collect()[0]["DAU"]
```

### 7-1. daily revenue 와 daily paying user 정보를 이용하여 ARPU 를 구하세요
```text
지표정의 : 유저 당 평균 발생 매출 금액
지표산식 : 총 매출 / 전체 유저 수 = DR / PU
입력형태 : Daily Revenue, Daily Paying User
출력형태 : number
```

### <span style="color:red"> [과제.7] </span> 아래의 <span style="color:green"> 과제) </span> 코멘트에 명시된 내용을 참고하여 각종 v_unknown / v_unknown 변수 값을 치환 합니다

In [ ]:
# 7. Average Revenue Per Paying User

# 과제 7) ARPPU (= DR / PU) 계산을 위한 v_unknown 를 각 각 다른 변수로 치환하세요
print("+------------------+")
print("|            ARPPU |")
print("+------------------+")
print("|        {} |".format(v_unknown / v_unknown))
print("+------------------+")

# 8. 디멘젼 테이블을 생성하세요
> 디멘젼 테이블을 스키마는 아래와 같습니다.

| 컬럼명 | 컬럼타입 | 설명 |
| - | - | - |
| d_uid | integer | 아이디 |
| d_name | string | 이름 |
| d_pamount | integer | 누적 구매 금액 |
| d_pcount | integer | 누적 구매 횟수 |
| d_acount | integer | 누적 접속 횟수 |
| d_first_purchase | string | 최초 구매 일시 |
| dt | string | 유저아이디 |

### 8-1. 서비스 오픈 첫 날이므로 오늘 로그만 이용하여 전체 유저아이디와 접속 횟수를 구합니다
```text
지표정의 : 이용자 누적 상태 정보
지표산식 : 오늘까지 접속한 모든 유저의 정보를 저장하는 테이블
입력형태 : user, purchase, access
출력형태 : 아래와 같이 설계합니다
```

### <span style="color:red"> [과제.8-1] </span> 아래의 <span style="color:green"> 과제) </span> 코멘트에 명시된 내용을 참고하여 countOfAccess 집계 질의문을 완성합니다

In [ ]:
access.printSchema()

# 과제 8-1) countOfAccess 는 access 테이블로부터 a_id 가 'login' 인 a_uid 값의 빈도수를 group by a_uid 집계를 통해 구하시오
countOfAccess = "select a_uid, <집계함수> as a_count from access where a_id = 'login' <집계구문>" 

uids = spark.sql(countOfAccess)
uids.show()

### 8-2. 일 별 이용자 별 총 매출 금액과, 구매 횟수를 추출합니다

### <span style="color:red"> [과제.8-2] </span> 아래의 <span style="color:green"> 과제) </span> 코멘트에 명시된 내용을 참고하여 sumOfCountAndAmount 집계 질의문을 완성합니다

In [ ]:
purchase.printSchema()

# 과제 8-2) sumOfCountAndAmount 는 purchase 테이블로 부터 p_uid 별 매출 횟수(count)와, 매출 금액의 합(sum)을 구하는 집계 쿼리를 생성 하시오
sumOfCountAndAmount = "select p_uid, <합-집계함수> as p_amount, <카운트-집계함수> as p_count from purchase <집계구문>" 

amts = spark.sql(sumOfCountAndAmount)
amts.show()

### 8-3. 이용자 정보와 구매 정보와 조인합니다
### <span style="color:red"> [과제.8-3] </span> 아래의 <span style="color:green"> 과제) </span> 코멘트에 명시된 내용을 참고하여 joinHow 조인 조건문을 작성합니다

In [ ]:
joinCondition = uids.a_uid == amts.p_uid

# 과제 8-3) Left Outer Join 연산을 위한 키워드를 입력합니다
joinHow = "<조인조건>"

dim1 = uids.join(amts, joinCondition, joinHow).sort(uids["a_uid"].asc())
dim1.printSchema()
dim1.show()

### 8-4. 고객 정보를 추가합니다
### <span style="color:red"> [과제.8-4] </span> 아래의 <span style="color:green"> 과제) </span> 코멘트에 명시된 내용을 참고하여 joinCondition 조인 조건을 작성합니다

In [ ]:
# 과제 8-4) joinCondition 는 dim1 의 a_uid 컬럼과 user 의 u_id 컬럼이 일치하는 조건을 작성합니다
joinCondition = ""

joinHow = "left_outer"

dim2 = dim1.join(user, joinCondition, joinHow)
dim2.printSchema()
dim2.show()

### 8-5. 중복되는 ID 컬럼은 제거하고, 숫자 필드에 널값은 0으로 기본값을 넣어줍니다
### <span style="color:red"> [과제.8-5] </span> 아래의 <span style="color:green"> 과제) </span> 코멘트에 명시된 내용을 참고하여 fillDefaultValue 의 내용을 완성합니다

In [ ]:
dim3 = dim2.drop("p_uid", "u_id")

# 과제 8-5) p_amount, p_count 필드의 기본값을 0으로 넣도록 작성합니다
fillDefaultValue = {"p_amount":0, "<기본값입력구문>":""}

dim4 = dim3.na.fill(fillDefaultValue)
dim4.printSchema()
dim4.show()

### 8-6. 생성된 유저 테이블을 재사용 가능하도록 컬럼 명을 변경합니다
### <span style="color:red"> [과제.8-6] </span> 아래의 <span style="color:green"> 과제) </span> 코멘트에 명시된 내용을 참고하여 withColumnRenamed 구문을 작성합니다

In [ ]:
dim4.printSchema()

dim5 = (
    dim4
    .withColumnRenamed("a_uid", "d_uid")
    .withColumnRenamed("a_count", "d_acount")
    .withColumnRenamed("p_amount", "d_pamount")
    .withColumnRenamed("p_count", "d_pcount")
    .withColumnRenamed("u_name", "d_name")
    # 과제 8-6) u_gender 컬럼을 d_gender 컬럼으로 rename 하는 구문을 추가합니다
    .drop("a_uid", "a_count", "p_amount", "p_count", "u_name", "u_gender")
    .select("d_uid", "d_name", "d_gender", "d_acount", "d_pamount", "d_pcount")
)

dim5.show()

### 8-7. 최초 구매 유저 정보를 추가합니다
```bash
# 아래의 구문을 통해 특정 컬럼이 널인 경우의 값을 선택할 수 있습니다 
expr("case when column1 is null then column2 else column1 end")
```

### <span style="color:red"> [과제.8-7] </span> 아래의 <span style="color:green"> 과제) </span> 코멘트에 명시된 내용을 참고하여 selectFirstPurchaseTime 구문을 완성합니다

In [ ]:
# 과제 8-7) selectFirstPurchaseTime 는 하루에 여러번 구매가 있을 수 있으므로 가장 먼저 구매한 일시를 min 함수를 p_time 의 최소값을 구합니다
selectFirstPurchaseTime = "select p_uid, <최소값함수> as p_time from purchase <집계구문>"

first_purchase = spark.sql(selectFirstPurchaseTime)
dim6 = dim5.withColumn("d_first_purchase", lit(None))

exprFirstPurchase = expr("case when d_first_purchase is null then p_time else d_first_purchase end")

dimension = dim6.join(first_purchase, dim5.d_uid == first_purchase.p_uid, "left_outer") \
.withColumn("first_purchase", exprFirstPurchase) \
.drop("d_first_purchase", "p_uid", "p_time") \
.withColumnRenamed("first_purchase", "d_first_purchase")

dimension.printSchema()
dimension.show()

### 8-8. 오늘 디멘젼 테이블을 dim_users/dt=20201025 경로에 저장합니다.
### <span style="color:red"> [과제.8-8] </span> 아래의 <span style="color:green"> 과제) </span> 코멘트에 명시된 내용을 참고하여 today_dimension 저장경로를 입력합니다

In [ ]:
# 과제 8-8) dim_users/dt=20201025 경로에 저장합니다
today_dimension = ""

dimension.write.mode("overwrite").parquet(today_dimension)

### 8-9. 오늘의 주요 지표를 출력합니다

### <span style="color:red"> [과제.8-9] </span> 실행 결과가 아래와 같은지 확인합니다

In [ ]:
# 과제 8-9) 최종 결과가 아래의 값과 일치하는지 확인합니다
# DAU:5, PU:4, DR: 12200000, ARPU: 2440000.0, ARPPU: 3050000.0
print("+------------------+")
print("|              DAU |")
print("+------------------+")
print("|                {} |".format(v_dau))
print("+------------------+")
print("+------------------+")
print("|               PU |")
print("+------------------+")
print("|                {} |".format(v_pu))
print("+------------------+")
print("+------------------+")
print("|                DR |")
print("+------------------+")
print("|        {} |".format(v_dr))
print("+------------------+")
print("+------------------+")
print("|             ARPU |")
print("+------------------+")
print("|        {} |".format(v_dr / v_dau))
print("+------------------+")
print("+------------------+")
print("|            ARPPU |")
print("+------------------+")
print("|        {} |".format(v_dr / v_pu))
print("+------------------+")